# Floating Point Binary Representation

Consider the case of 64-bit floating point binary representation, rather than the 32-bit floating point binary representation.

In [1]:
# convert base 10 number to binary
def convert_int_to_binary(num):
    res = ""

    while num > 0:
        digit = num % 2
        # print(num, digit)
        res = str(digit) + res
        num = int(num / 2)

    return res

# convert fractional number to binary
def convert_fraction_to_binary(num, precision=40):
    res = ""
    iter_count = 0  
    while num > 0 and precision > 0:
        num = num * 2
        if num >= 1:
            res += "1"
            num -= 1
        else:
            res += "0"
        precision -= 1
        #print(f"num = {num}  res = {res}  iter_count = {iter_count}    precision = {precision}")
        iter_count += 1
    return res

# convert a decimal number to IEEE 754 binary
def get_fp_binary_representation(n):
    # specify 48 decimal places of precision ... given that we will take 23 for the mantissa, this means we will have far more than we need
    p = 48

    # Step 1:  split the number into two parts - both strings
    front, back = str(n).split('.')

    # Step 2:  convert the part in front of the decimal to binary ... take the sign into account
    if int(front) < 0:
        sign = "-"
        i_front = -int(front)
    else:
        sign = ""
        i_front = int(front)

    if i_front == 0:
        front_bin = "0"
    else:
        front_bin = convert_int_to_binary(i_front)

    # Step 3:  convert the part after the decimal to binary
    divisor = 10**len(back)
    back_bin = convert_fraction_to_binary(float(back)/divisor, p)

    # Step 4:  add the strings together and print the result
    bin_result = sign + front_bin + "." + back_bin
    print(f"The binary representation of the {n} is {bin_result}")

    # Step 5:  Determine the exponent and mantissa
    if front_bin == "0":
        exponent = 0
        keep_going = True
        while keep_going:
            # print (back_bin[-exponent],exponent)
            if back_bin[-exponent] == "1":
                keep_going = False
                exponent = exponent + 1
            exponent = exponent - 1
        exponent = exponent - 1

        # print(exponent)
        back_bin = back_bin[-exponent:]
        mantissa_truncated = back_bin
    else:
        exponent = len(front_bin)-1
        mantissa = front_bin[1:] + back_bin
        mantissa_truncated = mantissa[0:23]

    true_result = sign + "1." + mantissa_truncated + " x 2^(" + str(exponent) + ")"
    print(f"The binary scientific notation representation is {true_result}")

    # Step 6:  Convert to 32-bit floating point representation
    if int(front) < 0:
        bit1 = "1"
    else:
        bit1 = "0"

    exp = int(exponent)+127
    exp_binary_rep = convert_int_to_binary(exp)
    if len(exp_binary_rep) < 8:
        exp_binary_rep = "0" + exp_binary_rep

    if len(mantissa_truncated) < 23:
        mantissa_truncated = mantissa_truncated + (23-len(mantissa_truncated))*"0"

    if len(mantissa_truncated) > 23:
        mantissa_truncated = mantissa_truncated[0:23]

    res = bit1 + "|" + exp_binary_rep + "|" + mantissa_truncated
    return res